# Prototype explanations

This script generates the typical prototype explanations for a prediction.

1) make sure to run the explanation script to generate a json file containing the nearest images in the training set for each prototype.

In [ ]:
import torch
import json
import os
import skimage as ski
from quanproto.metrics import helpers

from quanproto.utils.workspace import EXPERIMENTS_PATH, DATASET_DIR
from quanproto.evaluation import folder_utils as eval
from quanproto.dataloader.contrastivity_segmentation import test_dataloader
from quanproto.explanations.config_parser import load_model
from torch.nn.functional import max_pool2d
from quanproto.utils.vis_helper import save_image_mask
from quanproto.utils.vis_helper import save_image


In [ ]:

experiment_config = {
    "experiment_dir": f"{EXPERIMENTS_PATH}/ProtoMask/cub200",
    "dataset_dir": DATASET_DIR,
    "model": "protomask",
    "run": "banana-sunset-583",
    "explanation": "prp",
    "train_phase": "fine_tune",
    "crop": True,
}

test_image_idx = 970 # index of the image to explain


In [ ]:
# info from all runs in the experiment dir
run_info = eval.get_run_info(experiment_config)
run_info = run_info[experiment_config["run"]]
run_topk_info = eval.get_technique_results(
    experiment_config)["topk_prototype_images"]

with open(run_topk_info[experiment_config["run"]], "r") as f:
    topk_info = json.load(f)

# load the config file
with open(run_info["config"], "r") as f:
    run_config = json.load(f)

# make sure to use the datasets on your computer and not the path that was used to train the model
run_config["dataset_dir"] = experiment_config["dataset_dir"]

dataloader = test_dataloader(run_config,crop=experiment_config["crop"], fill_background_method="original")

model = load_model(
    run_config,
    experiment_config["explanation"],
    run_info[experiment_config["train_phase"]],
)

model.cuda()
model.eval()

In [ ]:
def vis_prototypes(ids, dataloader, model, topk_info):
    """
    Visualize the prototypes for the given image ids.
    """
    # make the ids tensor with BxN into a list 
    for img_i in range(ids.shape[0]): # all images
        for proto_j in range(ids.shape[1]): # all prototypes
            proto_idx = ids[img_i][proto_j]
            prototype_info = topk_info[f"{proto_idx}"]

            img_ids = prototype_info["ids"]

            img_batch = torch.zeros((len(img_ids), 3, 224, 224))
            for proto_nearest_i, proto_nearest_id in enumerate(img_ids):
                # get the image tensor
                img, _ = dataloader.dataset.getitem_by_id(proto_nearest_id)
                img_batch[proto_nearest_i] = img

            img_batch = img_batch.cuda()

            with torch.no_grad():
                # make a Bx1 tensor with the prototype index
                prototype_idx = torch.tensor([proto_idx]).cuda()
                prototype_idx = prototype_idx.expand(
                    img_batch.shape[0], prototype_idx.shape[0]
                )
                saliency_maps = model.saliency_maps(img_batch, prototype_idx)
                saliency_masks = torch.stack(
                    [
                        helpers.percentile_mask(saliency_maps[b, 0])
                        for b in range(saliency_maps.shape[0])
                    ]
                ).unsqueeze(1)
                saliency_maps = saliency_maps * saliency_masks

                for save_img_i in range(img_batch.shape[0]):
                    # get the image tensor
                    img = img_batch[save_img_i]
                    # get the saliency maps for the image
                    saliency_map = saliency_maps[save_img_i][0]

                    # save the saliency map
                    save_image_mask(img, saliency_map, f"{experiment_config["experiment_dir"]}/topk_explain/train_nearest_img_{save_img_i}_prototype_{proto_idx}.png")

In [ ]:
def vis_test_images(ids, dataloader, model):
    """
    Visualize the test image for the given image ids.
    """
    num_masks = dataloader.dataset.num_masks
    # 1) get the image tensor from the dataloader
    mask_batch = torch.zeros((len(ids), num_masks, 3, 224, 224))
    uncropped_mask_batch = torch.zeros((len(ids), num_masks, 3, 224, 224))
    original_bboxes_batch = torch.zeros((len(ids), num_masks, 4))
    original_mask_size_batch = torch.zeros((len(ids), 2))
    for i, id in enumerate(ids):
        # get the image tensor
        (masks, uncropped_masks, original_bboxes, original_mask_size), label = dataloader.dataset[id]
        mask_batch[i] = masks
        uncropped_mask_batch[i] = uncropped_masks
        original_bboxes_batch[i] = original_bboxes
        original_mask_size_batch[i] = original_mask_size

    mask_batch = mask_batch.cuda()
    uncropped_mask_batch = uncropped_mask_batch.cuda()
    original_bboxes_batch = original_bboxes_batch
    original_mask_size_batch = original_mask_size_batch

    with torch.no_grad():

        # region TopK Prototypes ---------------------------------------------------------------
        _, similarity_maps, _ = model.explain(mask_batch)

        similarity_scores = (
            max_pool2d(similarity_maps, kernel_size=similarity_maps.shape[2:])
            .squeeze(-1)
            .squeeze(-1)
        )
        # get the top k similarity scores indices
        k = 5
        _, topk_indices = torch.topk(similarity_scores, k=k, dim=1)
        # endregion TopK Prototypes ---------------------------------------------------------
        saliency_maps, mask_ids = model.saliency_maps(
            mask_batch, uncropped_mask_batch, original_bboxes_batch, original_mask_size_batch, topk_indices
        )
        if model.explanation_type == "mask":
            saliency_masks = saliency_maps.clone().detach()
        if model.explanation_type == "prp":
            saliency_masks = torch.stack(
                [
                    torch.stack(
                        [helpers.percentile_mask(saliency_maps[b, i]) for i in range(k)]
                    )
                    for b in range(saliency_maps.shape[0])
                ]
            )
            saliency_maps = saliency_maps * saliency_masks
            print(f"saliency_maps shape: {saliency_maps.shape}")

        # save the saliency maps for the test images
        print(f"mask_batch shape: {mask_batch.shape}")
        for i in range(mask_batch.shape[0]):
            # save all masks for the image
            for j in range(mask_batch.shape[1]):
                # get the mask tensor
                mask = mask_batch[i][j]
                print(f"mask shape: {mask.shape}")
                save_image(mask, f"{experiment_config['experiment_dir']}/topk_explain/test_img_{i}_mask_{j}.png")
            # get the image tensor
            img = mask_batch[i][mask_ids[i]]
            print(f"img shape: {img.shape}")
            # get the saliency maps for the image
            saliency_map = saliency_maps[i]

            for j in range(saliency_map.shape[0]):
                # get the saliency map for the prototype
                saliency_map_j = saliency_map[j]
                # save the saliency map
                save_image_mask(img, saliency_map_j, f"{experiment_config["experiment_dir"]}/topk_explain/test_img_{i}_prototype_{j}.png")
    return topk_indices.detach().cpu().numpy()


In [ ]:
prototype_ids = vis_test_images([test_image_idx], dataloader, model)